# Fairness in Biometrics: a figure of merit to assess biometric verification system

## The role of Z-Norm



In [1]:
%matplotlib widget

#################################################
## PLAYGROUND: PLAY WITH THE VALUES OF VARIABLES BELOW TO CHANGE THE BEHAVIOUR OF THE FAIR VERIFICATION SYSTEM

ALPHA = 0.5


# RANGE OF DECISION THRESHOLDS USED IN THE DEVELOPMENT SET
TEN_POWER_OF_X = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001] # BASICALLY, THE FMR THRESHOLDS

# NUMBER OF IDENTITIES PER DEMOGRAPHIC
IDENTITIES_PER_DEMOGRAPHIC = 300

# REPRESENTS HOW A SAMPLES ARE SAMPLED
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE AVERAGE EUCLIDEAN DISTANCE BETWEEN EACH IDENTITY
# EACH IDENTITY IS SAMPLED IN THE CORNER OF A HYPERCUBE
DEMOGRAPHIC_MEAN_DISTANCE = [0.8, 1.5, 2.3]
# EACH COLUMN REPRESENTS ONE DEMOGRAPHIC.
# THE VALUES REPRESENTS THE STANDARD DEVIATIONS OF THE SAMPLES SAMPLED IN THE CORNER OF A HYPERCUBE
DEMOGRAPHIC_STD = [0.1, 0.2, 0.3]
#################################################



import plots
import experiments
import scoring
import fairness_metric
import numpy as np


# fetching the samples
impostors_per_demographic_dev_fair, genuines_per_demographic_dev_fair, \
impostors_per_demographic_eval_fair, genuines_per_demographic_eval_fair = \
experiments.generate_uncalibrated_experiment(DEMOGRAPHIC_DISPLACEMENTS=DEMOGRAPHIC_MEAN_DISTANCE,
                                             DEMOGRAPHIC_STD=DEMOGRAPHIC_STD,
                                             DEV_EVAL_IDENTITIES_PER_DEMOGRAPHIC=IDENTITIES_PER_DEMOGRAPHIC)

n_demographics_fair = len(DEMOGRAPHIC_MEAN_DISTANCE)

# computing TAU USING THE ZEROTH-EFFORT IMPOSTORS from the development set
taus_fair = scoring.compute_thresholds(impostors_per_demographic_dev_fair, TEN_POWER_OF_X)

# Plot Figure 1 using data from evaluation set
plots.plot_cohort_box_plot(impostors_per_demographic_eval_fair, 
                     genuines_per_demographic_eval_fair, 
                     [f"{i}" for i in range(n_demographics_fair)],
                     thresholds=taus_fair,
                     far_thresholds=[10**(i+1) for i,_ in enumerate(TEN_POWER_OF_X)]
                    )

plots.compute_FMR_FNMR_FDR(impostors_per_demographic_eval_fair,
                           genuines_per_demographic_eval_fair,
                           thresholds=taus_fair,
                           far_thresholds=TEN_POWER_OF_X,
                           tablefmt="plain")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

##############################################
################## FMR #######################
##############################################
       0.1    0.01    0.001    0.0001    1e-05    1e-06
 0  0.2255  0.0256   0.0024    0.0002        0        0
 1  0.0024  0        0         0             0        0
 2  0       0        0         0             0        0
##############################################
################## FNMR #######################
##############################################
       0.1    0.01    0.001    0.0001    1e-05    1e-06
 0  0       0        0         0        0        0
 1  0       0.006    0.0876    0.2471   0.4596   0.6482
 2  0.0016  0.5701   0.8576    0.9438   0.981    0.9909
##############################################
################## FDR #######################
##############################################
    0.1     0.01    0.001    0.0001    1e-05    1e-06
0.88645  0.70215     0.57     0.528   0.5095  0.50455


In [30]:
def generate_znorm_experiment(
    DEMOGRAPHIC_DISPLACEMENTS=[1, 1, 1],
    DEMOGRAPHIC_STD=[0.2, 0.2, 0.2],
    DEV_EVAL_IDENTITIES_PER_DEMOGRAPHIC=100,
    T_NORM_IDENTITIES_PER_DEMOGRAPHIC=25,
    Z_NORM_IDENTITIES_PER_DEMOGRAPHIC=25,
    SEED=10,
):

    (
        enroll_dev,
        probe_dev,
        enroll_eval,
        probe_eval,
        z_norm_samples,
        t_norm_samples,
    ) = experiments.generate_abstract_samples(
        DEMOGRAPHIC_DISPLACEMENTS=DEMOGRAPHIC_DISPLACEMENTS,
        DEMOGRAPHIC_STD=DEMOGRAPHIC_STD,
        DEV_EVAL_IDENTITIES_PER_DEMOGRAPHIC=DEV_EVAL_IDENTITIES_PER_DEMOGRAPHIC,
        T_NORM_IDENTITIES_PER_DEMOGRAPHIC=T_NORM_IDENTITIES_PER_DEMOGRAPHIC,
        Z_NORM_IDENTITIES_PER_DEMOGRAPHIC=Z_NORM_IDENTITIES_PER_DEMOGRAPHIC,
        SEED=SEED,
    )

    #impostors_per_demographic_dev_unfair, genuines_per_demographic_dev_unfair, \
    #impostors_per_demographic_eval_unfair, genuines_per_demographic_eval_unfair = \
    #generate_raw_scores(enroll_dev, probe_dev, enroll_eval, probe_eval)
    
    def compute_znorm_statistics(enroll_samples, z_norm):
        
        def stack_dictionary(z_norm):
            return np.vstack([z_norm[demographic][identity] for demographic in z_norm for identity in z_norm[demographic]])

        z_probes = stack_dictionary(z_norm)    
        
        z_statistics = {}
        for demographic in enroll_samples:
            z_statistics[demographic] = {}
            for identity in enroll_samples[demographic]:
                z_statistics[demographic][identity] = {}
                z_scores = scoring.compute_scores([enroll_samples[demographic][identity]],z_probes)[0]
                z_statistics[demographic][identity]["mu"] = np.mean(z_scores)
                z_statistics[demographic][identity]["std"] = np.std(z_scores)
        return z_statistics
        
    z_statistics = compute_znorm_statistics(enroll_dev, z_norm_samples)
    
    print(z_statistics)
    
                
generate_znorm_experiment()                

{0: {0: {'mu': -4.2131890326892005, 'std': 0.5219457242648694}, 1: {'mu': -4.106491150685073, 'std': 0.5447890633916114}, 2: {'mu': -4.152632804819067, 'std': 0.5237488565684416}, 3: {'mu': -4.206168604194692, 'std': 0.5673408652498316}, 4: {'mu': -4.511208138016518, 'std': 0.643013385595466}, 5: {'mu': -3.9234281687005272, 'std': 0.5821891563306566}, 6: {'mu': -4.221642116781441, 'std': 0.5586131950256825}, 7: {'mu': -4.066731198122868, 'std': 0.5817714811733453}, 8: {'mu': -4.389095587070601, 'std': 0.5608477422232961}, 9: {'mu': -4.114971969393636, 'std': 0.5654794730004989}, 10: {'mu': -4.261329312869845, 'std': 0.619872900415539}, 11: {'mu': -4.148492411617703, 'std': 0.6235370150392832}, 12: {'mu': -4.426430106446933, 'std': 0.7173326196152813}, 13: {'mu': -3.907993964507495, 'std': 0.6776595412728608}, 14: {'mu': -4.456683920793405, 'std': 0.7378103752517988}, 15: {'mu': -4.095068109373371, 'std': 0.6513366075579541}, 16: {'mu': -3.8288800317799225, 'std': 0.6148709044143156}, 1